In [11]:
import os
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import scipy.stats as stats
from scipy.stats import norm, ttest_ind

In [2]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

In [7]:
def get_sample_size_abs(epsilon, std, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    z_scores_sum_squared = (t_alpha + t_beta) ** 2
    sample_size = int(
        np.ceil(
            z_scores_sum_squared * (2 * std ** 2) / (epsilon ** 2)
        )
    )
    return sample_size

In [8]:
def get_sample_size_arb(mu, std, eff=1.01, alpha=0.05, beta=0.2):
    epsilon = (eff - 1) * mu

    return get_sample_size_abs(epsilon, std=std, alpha=alpha, beta=beta)

def check_ttest(a, b, alpha=0.05):
    """Тест Стьюдента. Возвращает 1, если отличия значимы."""
    _, pvalue = ttest_ind(a, b)
    return int(pvalue < alpha)

# Задание 1

Допустим, мы хотим провести эксперимент, в который попадают клиенты, совершившие покупку во время эксперимента.  

* Метрика — средняя выручка с пользователя за время эксперимента;
* Продолжительность — одна неделя;
* Уровень значимости — 0.05;
* Допустимая вероятность ошибки II рода — 0.1;
* Ожидаемый эффект — 20 рублей.


Оцените необходимый размер групп по данным о покупках за неделю с 21 по 28 февраля. Обратим внимание, что в выборку попадают события из полуинтервала [datetime(2022, 2, 21), datetime(2022, 2, 28)).

In [3]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [42]:
df_sales_hist = df_sales[(df_sales['date'] >= datetime(2022, 2, 21)) & (df_sales['date'] < datetime(2022, 2, 28))].copy()
df_sales_hist = df_sales_hist.groupby(['user_id'], as_index=False)['price'].sum()

In [43]:
alpha = 0.05
beta = 0.1
effect = 20

mean = df_sales_hist['price'].mean()
std = df_sales_hist['price'].std()

sample_size = get_sample_size_abs(effect, std, alpha=alpha, beta=beta)

print(f'samples size per group = {round(sample_size, -1)}')

samples size per group = 34570


# Задача 2

В прошлом задании получилось, что необходимый размер групп больше имеющихся данных за одну неделю. Какой минимальный эффект мы можем отловить с теми же вероятностями ошибок на данных
с 21 по 28 февраля?

In [44]:
def get_minimal_determinable_effect(std, sample_size, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (2 * (std ** 2)) ** 0.5
    mde = (t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(sample_size)
    return mde

In [56]:
hist_sample_size = df_sales_hist['user_id'].nunique() / 2
print(f'Минимальный эффект = {round(get_minimal_determinable_effect(std, hist_sample_size, alpha=alpha, beta=beta))}')

Минимальный эффект = 33
